Housing Values in Suburbs of Boston
https://www.kaggle.com/c/boston-housing <br>
根据13个指标预测波士顿房价中位数
* The medv variable is the target variable.(房价的中位数为预测目标)
* The Boston data frame has 506 rows and 14 columns.(shape: 506*14)
* The scoring is based on the RMSE, that is:
$RMSE(y) = \sqrt{\sum_{i=1}^n (y_i - \hat{y}_i)^2}$ (均方误差的开方)


In [1]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 6us/step


In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(y_train)

(404, 13) (404,)
(102, 13) (102,)
[ 15.2  42.3  50.   21.1  17.7  18.5  11.3  15.6  15.6  14.4  12.1  17.9
  23.1  19.9  15.7   8.8  50.   22.5  24.1  27.5  10.9  30.8  32.9  24.
  18.5  13.3  22.9  34.7  16.6  17.5  22.3  16.1  14.9  23.1  34.9  25.
  13.9  13.1  20.4  20.   15.2  24.7  22.2  16.7  12.7  15.6  18.4  21.
  30.1  15.1  18.7   9.6  31.5  24.8  19.1  22.   14.5  11.   32.   29.4
  20.3  24.4  14.6  19.5  14.1  14.3  15.6  10.5   6.3  19.3  19.3  13.4
  36.4  17.8  13.5  16.5   8.3  14.3  16.   13.4  28.6  43.5  20.2  22.
  23.   20.7  12.5  48.5  14.6  13.4  23.7  50.   21.7  39.8  38.7  22.2
  34.9  22.5  31.1  28.7  46.   41.7  21.   26.6  15.   24.4  13.3  21.2
  11.7  21.7  19.4  50.   22.8  19.7  24.7  36.2  14.2  18.9  18.3  20.6
  24.6  18.2   8.7  44.   10.4  13.2  21.2  37.   30.7  22.9  20.   19.3
  31.7  32.   23.1  18.8  10.9  50.   19.6   5.   14.4  19.8  13.8  19.6
  23.9  24.5  25.   19.9  17.2  24.6  13.5  26.6  21.4  11.9  22.6  19.6
   8.5  23.7  23.1  2

In [5]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean)/std
x_test = (x_test - mean)/std

In [19]:
def build_model():
    model = Sequential()
    model.add(Dense(64,input_shape=(13,),activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(), loss='mse', metrics=['mae'])#平均绝对误差
    return model

<img src="imgs/交叉验证.png">

In [27]:
folds = 4
nb_samples = len(x_train) // folds
nb_epochs = 100
all_scores = []
for i in range(folds):
    #验证集
    val_data = x_train[i*nb_samples : (i+1)*nb_samples]
    val_targets = y_train[i*nb_samples : (i+1)*nb_samples]

    #训练集
#   X_train = np.concatenate((x_train[ : i*nb_samples], x_train[(i+1)*nb_samples : ]), axis=0)
    X_train = np.vstack((x_train[ : i*nb_samples], x_train[(i+1)*nb_samples : ]))
#   Y_train = np.concatenate((y_train[ : i*nb_samples], y_train[(i+1)*nb_samples : ]), axis=0)
    Y_train = np.hstack((y_train[ : i*nb_samples], y_train[(i+1)*nb_samples : ]))
    
    model = build_model()
    model.fit(X_train, Y_train, epochs=nb_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mse)
    #MSE-均方误差  MAE-平均绝对误差
    print('第 %d 折, MSE: %f MAE:%f'%(i+1,val_mse,val_mae))

第 1 折, MSE: 9.047565 MAE:2.179007
第 2 折, MSE: 12.376006 MAE:2.494610
第 3 折, MSE: 15.710650 MAE:2.613543
第 4 折, MSE: 11.860395 MAE:2.366552


In [28]:
np.sqrt(np.mean(all_scores))

3.4998077363123397

In [24]:
model = build_model()
model.fit(x_train, y_train, epochs=80, batch_size=16, verbose=0)

In [25]:
test_mse, test_mae = model.evaluate(x_test, y_test)

102/102 [==============================] - 0s 1ms/step


In [26]:
print(np.sqrt(test_mse))

4.17893866342
